In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

https://www.yuque.com/sdutzhou/durg6g/gvziv4u9cxvta5k5?singleDoc# 《深度学习与建模专题简介》

https://www.yuque.com/sdutzhou/durg6g/ccksn7esdremwn23?singleDoc#cldRW


通过百度网盘分享的文件：boston_housing.csv
链接：https://pan.baidu.com/s/1fLWH-EmAjtqktg7cx3Cbbw?pwd=6868 
 

In [ ]:
getwd()
 

In [ ]:
#(1) 创建任务
library(tidyverse)
library(mlr3verse)

boston = read_csv("/kaggle/input/boston-housing/boston_housing.csv",
                  col_types = cols(chas = "f", town = "f"))
# 读取数据，同时设置 chas 和 town 列按因子类型读取：
glimpse(boston)

#用as_task_regr()函数创建回归任务：
task =as_task_regr(boston,target= "cmedv")
task

autoplot(task) # 可视化任务

In [ ]:
 

# From CRAN:
install.packages("mlr3verse")

# From Github:
#remotes::install_github("mlr-org/mlr3verse")

In [ ]:
mlr3verse_info()

In [ ]:
prep = po("encode", method = "treatment",
          affect_columns = selector_name("chas")) %>>%
  po("encodeimpact",
     affect_columns = selector_name("town"))

# town 水平值过于分散，采用效应编码
# chas 是两水平因子，处理成虚拟变量





In [ ]:
task = prep$train(task)[[1]]


In [ ]:
set.seed(123)
# 保证结果可重现
split = partition(task, ratio = 0.8) # 默认 stratify=TRUE
#用 partition() 函数对任务做划分，默认按因变量分层，返回列表包含训练集索引和测试集索引。
#(1)选择学习器
lrn_ridge = lrn("regr.glmnet", alpha= 0) #需安装glmnet包
lrn_ridge

#查看模型的所有超参数及默认值（结果略）
lrn_ridge$param_set

In [ ]:
search_space = ps(s=p_dbl(0.001, 2))
at = auto_tuner(
  tuner=tnr("random_search"),
  learner= lrn_ridge,
  resampling= rsmp("cv", folds= 3L), #3折交叉验证
  measure= msr("regr.rmse"), #评估指标选rmse
  search_space=search_space,
  term_evals= 10) #计算10次终止




In [ ]:
#在训练集上启动自动调参过程
set.seed(123)
at$train(task, row_ids= split$train)
#查看最优超参数
at$tuning_result

In [ ]:
#可视化超参数变化对模型性能的影响
autoplot(at$tuning_instance)



In [ ]:
#训练模型
model = at$learner$model


In [ ]:
#提取岭回归模型系数，绘制岭迹图：
coef(model, s = at$tuning_result$s)
plot(model, xvar = "lambda", label = TRUE)




In [ ]:
#或者直接用 autoplot() 绘图：
autoplot(at$learner, type = "ggfortify")




In [ ]:
#在测试集上做预测：
pred_ridge = at$predict(task, row_ids = split$test)
pred_ridge





In [ ]:
#模型评估
#计算测试集上的泛化性能：均方根误差与 𝑅 方
pred_ridge$score(msrs(c("regr.rmse", "regr.rsq")))



In [ ]:
#6)预测新数据
newdata =task$data()[1:5,-1] #作为新数据
at$predict_newdata(newdata)